In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
#CSV파일 로드
data = pd.read_csv("/content/drive/MyDrive/lstm_data.csv")

In [ ]:
#CSV파일을 레이블과 입력 데이터로 분할
labels = data.iloc[:, 0].values
input_data = data.iloc[:, 1:].values


# 데이터 크기 확인
# 행의 개수
num_rows = input_data.shape[0]
# 열의 개수 
num_cols = input_data.shape[1]  

print(num_rows)
print(num_cols)
print(input_data)

70
10
[[ 0.1   0.1  -0.27  0.05  0.01  0.1  -0.04 -0.75  0.3   1.92]
 [ 0.05  0.01  0.1  -0.04 -0.75  0.3   1.92 -0.92 -0.99  0.74]
 [ 2.41  0.4   0.31  0.19  0.23  0.07 -0.1   0.12 -1.38  4.75]
 [-2.17  0.31 -0.23 -0.56 -0.19  0.55  0.83  0.84  0.66  0.67]
 [ 0.67  0.66  2.46  0.53  1.29  0.21  0.21  0.73 -1.59  0.19]
 [-0.22 -0.65 -0.04 -0.2  -0.93  6.9  -2.18 -1.31 -0.86  1.21]
 [-0.53  0.31  0.42  0.6   2.2  -1.09 -0.6  -0.15 -0.18  0.26]
 [-0.15 -0.18  0.26  1.26  0.7   0.12  2.63 -0.18 -2.34 -2.74]
 [ 0.3  -0.09  0.16  0.14 -0.27 -0.26 -0.1   0.19  0.34  0.07]
 [-0.01  0.12 -0.11  0.64 -0.07  0.03  0.03 -0.25  0.04 -0.02]
 [-0.83  0.07 -0.46  6.16 -2.15 -0.28 -0.52  0.48  0.05  0.08]
 [-0.1   0.2  -0.02  0.01 -0.53 -0.26 -0.14 -0.05  0.08  0.13]
 [ 0.19  0.09 -0.12 -0.    0.06 -0.08  0.19 -0.15  0.12  0.24]
 [ 0.19 -0.15  0.12  0.24 -0.06 -0.07 -0.93  0.14 -0.39  0.03]
 [-0.65 -0.06  0.06  0.35 -0.06  0.07 -0.06 -0.1  -0.1   0.15]
 [-0.16 -0.39  0.16  0.16 -0.12 -0.02 -0.11  0.01

In [ ]:
#입력된 데이터의 크기가 10이므로 time_steps = 10로 지정
time_steps = 10
input_sequences = []
target_labels = []

#time_steps의 크기만큼 데이터를 잘라 input_data는 (1,10,1)의 3차원 배열로, target_lebel은 (1,1)의 2차원 배열로 바꾸어주고 각각 input_sequences, target_labels에 넣어준다.
for i in range(num_rows - time_steps + 1):
    input_sequence = input_data[i:i+time_steps, 0].reshape((1, time_steps, 1))
    input_sequences.append(input_sequence)
    target_label = labels[i+time_steps-1].reshape((1,1))
    target_labels.append(target_label)

target_labels = np.array(target_labels)


print(input_sequences)
print(target_labels)


[array([[[ 0.1 ],
        [ 0.05],
        [ 2.41],
        [-2.17],
        [ 0.67],
        [-0.22],
        [-0.53],
        [-0.15],
        [ 0.3 ],
        [-0.01]]]), array([[[ 0.05],
        [ 2.41],
        [-2.17],
        [ 0.67],
        [-0.22],
        [-0.53],
        [-0.15],
        [ 0.3 ],
        [-0.01],
        [-0.83]]]), array([[[ 2.41],
        [-2.17],
        [ 0.67],
        [-0.22],
        [-0.53],
        [-0.15],
        [ 0.3 ],
        [-0.01],
        [-0.83],
        [-0.1 ]]]), array([[[-2.17],
        [ 0.67],
        [-0.22],
        [-0.53],
        [-0.15],
        [ 0.3 ],
        [-0.01],
        [-0.83],
        [-0.1 ],
        [ 0.19]]]), array([[[ 0.67],
        [-0.22],
        [-0.53],
        [-0.15],
        [ 0.3 ],
        [-0.01],
        [-0.83],
        [-0.1 ],
        [ 0.19],
        [ 0.19]]]), array([[[-0.22],
        [-0.53],
        [-0.15],
        [ 0.3 ],
        [-0.01],
        [-0.83],
        [-0.1 ],
        [ 0.19]

In [ ]:
input_sequences = np.repeat(input_sequences, 70, axis=0)  # 입력 데이터의 샘플 수를 70으로 변경
target_labels = np.repeat(target_labels, 70, axis=0) # 레이블의 샘플 수를 70으로 변경

input_sequences = input_sequences.reshape(-1, time_steps, 1) # 입력 데이터의 형태를 shape=(1, time_steps, 1)로 재설정

# 변환된 데이터 확인
print(f"입력 데이터 shape: {input_sequences[0].shape[0]}")
print(f"레이블 shape: {target_labels[0].shape[0]}")


입력 데이터 shape: 10
레이블 shape: 1


In [ ]:
# 모델 구조
model = tf.keras.Sequential([
    # 입력층 : 64개의 노드로 구성된 LSTM 레이어
    tf.keras.layers.LSTM(64, input_shape=(time_steps, 1),return_sequences=True),
    # 중간층(은닉층) : 64개 노드로 구성된 LSTM 레이어
    tf.keras.layers.LSTM(64),
    # 출력층 : 1개의 노드로 구성된 레이어, 활성화함수는 Sigmoid
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
print(f"입력 데이터 개수: {len(input_sequences)}")
print(f"레이블 개수: {len(target_labels)}")

print(input_sequences)
print(target_labels)


입력 데이터 개수: 260470
레이블 개수: 260470
[[[ 0.1 ]
  [ 0.05]
  [ 2.41]
  ...
  [-0.15]
  [ 0.3 ]
  [-0.01]]

 [[ 0.1 ]
  [ 0.05]
  [ 2.41]
  ...
  [-0.15]
  [ 0.3 ]
  [-0.01]]

 [[ 0.1 ]
  [ 0.05]
  [ 2.41]
  ...
  [-0.15]
  [ 0.3 ]
  [-0.01]]

 ...

 [[-0.73]
  [ 0.27]
  [ 0.89]
  ...
  [-0.12]
  [ 0.3 ]
  [-6.27]]

 [[-0.73]
  [ 0.27]
  [ 0.89]
  ...
  [-0.12]
  [ 0.3 ]
  [-6.27]]

 [[-0.73]
  [ 0.27]
  [ 0.89]
  ...
  [-0.12]
  [ 0.3 ]
  [-6.27]]]
[[[0]]

 [[0]]

 [[0]]

 ...

 [[1]]

 [[1]]

 [[1]]]


In [ ]:
#학습 진행
history = model.fit(input_sequences, target_labels, epochs=100, batch_size=32)

Epoch 1/100
8140/8140 [==============================] - 80s 10ms/step - loss: 0.0236 - accuracy: 0.9868
Epoch 2/100
8140/8140 [==============================] - 77s 9ms/step - loss: 4.1313e-07 - accuracy: 1.0000
Epoch 3/100
8140/8140 [==============================] - 77s 9ms/step - loss: 1.2559e-08 - accuracy: 1.0000
Epoch 4/100
8140/8140 [==============================] - 77s 9ms/step - loss: 4.7102e-09 - accuracy: 1.0000
Epoch 5/100
8140/8140 [==============================] - 77s 9ms/step - loss: 3.5309e-09 - accuracy: 1.0000
Epoch 6/100
8140/8140 [==============================] - 76s 9ms/step - loss: 0.0065 - accuracy: 0.9989
Epoch 7/100
8140/8140 [==============================] - 77s 9ms/step - loss: 1.7303e-06 - accuracy: 1.0000
Epoch 8/100
8140/8140 [==============================] - 77s 9ms/step - loss: 3.5565e-08 - accuracy: 1.0000
Epoch 9/100
8140/8140 [==============================] - 77s 9ms/step - loss: 7.0914e-09 - accuracy: 1.0000
Epoch 10/100
8140/8140 [===========

In [ ]:
#학습된 모델의 정확도(Accuracy)와 손실값(Loss) 확인 
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f"손실값: {loss}")
print(f"정확도: {accuracy}")


손실값: [0.02356025204062462, 4.1313393239761353e-07, 1.2558925455152803e-08, 4.7101695876961e-09, 3.530860714917594e-09, 0.00645297160372138, 1.7302525066043017e-06, 3.556518635150496e-08, 7.091388365410012e-09, 5.66556712655597e-09, 5.684940518335679e-09, 5.866287899891631e-09, 5.806749303616243e-09, 5.4937512317110304e-09, 5.778151290769529e-09, 5.78176173604561e-09, 5.799627444957878e-09, 5.554506188332198e-09, 5.7367839367827855e-09, 5.727976315483829e-09, 5.559522620046664e-09, 5.523664636797321e-09, 5.634146482691449e-09, 5.727822660617221e-09, 5.622342147404424e-09, 5.594689600485481e-09, 5.641165312653129e-09, 5.4959783390984285e-09, 5.20331555620146e-09, 5.558071336508874e-09, 5.526965551894136e-09, 5.5199929072102805e-09, 5.363822719317568e-09, 5.537295066915249e-09, 5.5281583755117936e-09, 5.473766329089358e-09, 5.509396494574048e-09, 5.322953189335067e-09, 5.467708508177793e-09, 5.1938831013842446e-09, 5.316814988276519e-09, 4.9405017854553535e-09, 5.496233690394092e-09, 5.49

In [ ]:
#학습된 모델을 h5파일로 저장 
model.save("Acc_LSTM_2.h5")

In [ ]:
#h5파일을 구글 드라이브에 저장
%cp "Acc_LSTM_2.h5" "/content/drive/MyDrive"